### 1. Load the houseprices data from Thinkful's database.

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse


import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

In [14]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
housing_df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [15]:
housing_df.head()
# question 1 is done

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


### 2. Do data cleaning, exploratory data analysis, and feature engineering. You can use your previous work in this module. But make sure that your work is satisfactory.

In [16]:
housing_df.isnull().sum() * 100/housing_df.isnull().count()

id                0.000000
mssubclass        0.000000
mszoning          0.000000
lotfrontage      17.739726
lotarea           0.000000
street            0.000000
alley            93.767123
lotshape          0.000000
landcontour       0.000000
utilities         0.000000
lotconfig         0.000000
landslope         0.000000
neighborhood      0.000000
condition1        0.000000
condition2        0.000000
bldgtype          0.000000
housestyle        0.000000
overallqual       0.000000
overallcond       0.000000
yearbuilt         0.000000
yearremodadd      0.000000
roofstyle         0.000000
roofmatl          0.000000
exterior1st       0.000000
exterior2nd       0.000000
masvnrtype        0.547945
masvnrarea        0.547945
exterqual         0.000000
extercond         0.000000
foundation        0.000000
                   ...    
bedroomabvgr      0.000000
kitchenabvgr      0.000000
kitchenqual       0.000000
totrmsabvgrd      0.000000
functional        0.000000
fireplaces        0.000000
f

From the above list, we can see that a few of our features have many null values. For each feature missing more than 6% of its values, we will replace its nulls.

Lot Frontage is  “Linear feet of street connected to property". Null values for this field come from properties that do not have this property, so a value of 0 can be used to replace Null values.

In [17]:
housing_df['lotfrontage'].fillna(0,inplace=True)

The Alley column lists the "Type of alley access", so the Null values for this column are from houses that do not have alleys. The Null values can be replaced with "No" so that later a dummy variable can be constructed if necessary.

In [18]:
housing_df['alley'].fillna('No',inplace=True)

Fireplacequ lists the "Fireplace quality", so houses without fireplaces will have Null values. These Null values can be replaced with "No", so that a dummy variable can be constructed later if necessary.

In [19]:
housing_df['fireplacequ'].fillna('No',inplace=True)

Poolqc lists "Pool quality", so houses that have no pool will have Null values. These Null values can be replaced with "No", so that a dummy variable can be constructed later if necessary.

In [20]:
housing_df['poolqc'].fillna('No',inplace=True)

Fence lists "Fence quality", so houses that have no fences will have Null values. These Null values can be replaced with "No", so that a dummy variable can be constructed later if necessary.

In [21]:
housing_df['fence'].fillna('No',inplace=True)

Miscfeature lists "Miscellaneous feature not covered in other categories", so there will be Null values for houses without features that need to be covered in this column. These Null values can be replaced with "No", so that a dummy variable can be constructed later if necessary.

In [22]:
housing_df['miscfeature'].fillna('No',inplace=True)

From previous work on this dataset, we know that we can make a reliable model using certain features. These features will be used below.

mszoning is an important feature, but it is categorical, so dummy variables must be constructed. Similarly, street is useful but categorical, so dummy variables must be constructed.

In [23]:
housing_df = pd.concat([housing_df,pd.get_dummies(housing_df.mszoning, prefix="mszoning", drop_first=True)], axis=1)
housing_df = pd.concat([housing_df,pd.get_dummies(housing_df.street, prefix="street", drop_first=True)], axis=1)
dummy_column_names = list(pd.get_dummies(housing_df.mszoning, prefix="mszoning", drop_first=True).columns)
dummy_column_names = dummy_column_names + list(pd.get_dummies(housing_df.street, prefix="street", drop_first=True).columns)

Two new variables will be created from the already-existent variables in the dataset. totalsf is the sum of the first floor, second floor, and basement square footage. int_over_sf is constructed from totalsf multiplied by overallqual.

In [26]:
housing_df['totalsf'] = housing_df['totalbsmtsf'] + housing_df['firstflrsf'] + housing_df['secondflrsf']

housing_df['int_over_sf'] = housing_df['totalsf'] * housing_df['overallqual']

The target variable, saleprice, is not normally distributed, so using a log transform will improve our model.

In [27]:
# Y is the target variable
# log transform the target variable to make it more normal
Y = np.log1p(housing_df['saleprice'])
# X is the feature set
X = housing_df[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalsf', 'int_over_sf'] + dummy_column_names]

The exploratory phase can now be concluded. For the model, overallqual, grlivarea, garagecars, garagearea, totalsf, int_over_sf, the mszoning dummies created above, and the street dummies created above will be used as predictors.

### 3. Now, split your data into train and test sets where 20% of the data resides in the test set.

In [28]:
X = sm.add_constant(X)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)

results = sm.OLS(y_train, X_train).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.832
Model:                            OLS   Adj. R-squared:                  0.831
Method:                 Least Squares   F-statistic:                     520.9
Date:                Wed, 04 Dec 2019   Prob (F-statistic):               0.00
Time:                        07:06:00   Log-Likelihood:                 463.99
No. Observations:                1168   AIC:                            -904.0
Df Residuals:                    1156   BIC:                            -843.2
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           9.9162      0.102     97.518      0.000       9.717      10.116
overallqual     0.1893      0.009     20.123      0.000       0.171       0.208
grlivarea     9.58e-05   1.89e-05      5.074      0.000    5.88e-05       0.000
garagecars      0.0779      0.015      5.244      0.000       0.049       0.107
garagearea      0.0001   5.04e-05      2.132      0.033    8.57e-06       0.000
totalsf         0.0003   2.58e-05     11.139      0.000       0.000       0.000
int_over_sf -2.572e-05   3.02e-06     -8.526      0.000   -3.16e-05   -1.98e-05
mszoning_FV     0.3911      0.065      6.055      0.000       0.264       0.518
mszoning_RH     0.2650      0.074      3.593      0.000       0.120       0.410
mszoning_RL     0.3879      0.060      6.481      0.000       0.270       0.505
mszoning_RM     0.2155      0.061      3.556      0.000       0.097       0.334
street_Pave    -0.0556      0.075     -0.744      0.457      -0.202       0.091
==============================================================================
Omnibus:                      350.711   Durbin-Watson:                   1.876
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2714.386
Skew:                          -1.167   Prob(JB):                         0.00
Kurtosis:                      10.094   Cond. No.                     5.33e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.33e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

The first model, using OLS, has an R-Squared of .832 AIC of -904, and BIC of -843. 83% of the variance in our data is explained by this model.

### 4. Build several linear regression models including Lasso, Ridge, or ElasticNet and train them in the training set. Use k-fold cross-validation to select the best hyperparameters if your models include one!

In [31]:
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV

In [35]:
# construct a range of alpha values to be used in the models.
# the best alpha value for each model will be reported.
alphas = [np.power(10.0,p) for p in np.arange(-10,40,1)]


# round 'em up
lasso_cv = LassoCV(alphas=alphas, cv=5)

lasso_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = lasso_cv.predict(X_train)
y_preds_test = lasso_cv.predict(X_test)

print(f"Best alpha value is: {lasso_cv.alpha_}")
print(f"R-squared of the model in training set is: {lasso_cv.score(X_train, y_train)}")
print("-----Test set statistics-----")
print(f"R-squared of the model in test set is: {lasso_cv.score(X_test, y_test)}")
print(f"Mean absolute error of the prediction is: {mean_absolute_error(y_test, y_preds_test)}")
print(f"Mean squared error of the prediction is: {mse(y_test, y_preds_test)}")
print(f"Root mean squared error of the prediction is: {rmse(y_test, y_preds_test)}")
print(f"Mean absolute percentage error of the prediction is: {np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100}")

Best alpha value is: 0.0001
R-squared of the model in training set is: 0.831939428704242
-----Test set statistics-----
R-squared of the model in test set is: 0.8226434437869412
Mean absolute error of the prediction is: 0.12624310826908416
Mean squared error of the prediction is: 0.029573434037677038
Root mean squared error of the prediction is: 0.17196928225028166
Mean absolute percentage error of the prediction is: 1.0552354946577736


In [33]:
# ruffles have ridges
ridge_cv = RidgeCV(alphas=alphas, cv=5)

ridge_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = ridge_cv.predict(X_train)
y_preds_test = ridge_cv.predict(X_test)

print(f"Best alpha value is: {ridge_cv.alpha_}")
print(f"R-squared of the model in training set is: {ridge_cv.score(X_train, y_train)}")
print("-----Test set statistics-----")
print(f"R-squared of the model in test set is: {ridge_cv.score(X_test, y_test)}")
print(f"Mean absolute error of the prediction is: {mean_absolute_error(y_test, y_preds_test)}")
print(f"Mean squared error of the prediction is: {mse(y_test, y_preds_test)}")
print(f"Root mean squared error of the prediction is: {rmse(y_test, y_preds_test)}")
print(f"Mean absolute percentage error of the prediction is: {np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100}")

Best alpha value is: 1.0
R-squared of the model in training set is: 0.8316364867222636
-----Test set statistics-----
R-squared of the model in test set is: 0.8203050076234274
Mean absolute error of the prediction is: 0.1267363733974108
Mean squared error of the prediction is: 0.029963358092979037
Root mean squared error of the prediction is: 0.1730992723640947
Mean absolute percentage error of the prediction is: 1.0596941230310684


In [34]:
elasticnet_cv = ElasticNetCV(alphas=alphas, cv=5)

elasticnet_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = elasticnet_cv.predict(X_train)
y_preds_test = elasticnet_cv.predict(X_test)

print(f"Best alpha value is: {elasticnet_cv.alpha_}")
print(f"R-squared of the model in training set is: {elasticnet_cv.score(X_train, y_train)}")
print("-----Test set statistics-----")
print(f"R-squared of the model in test set is: {elasticnet_cv.score(X_test, y_test)}")
print(f"Mean absolute error of the prediction is: {mean_absolute_error(y_test, y_preds_test)}")
print(f"Mean squared error of the prediction is: {mse(y_test, y_preds_test)}")
print(f"Root mean squared error of the prediction is: {rmse(y_test, y_preds_test)}")
print(f"Mean absolute percentage error of the prediction is: {np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100}")

Best alpha value is: 0.001
R-squared of the model in training set is: 0.8299654806803803
-----Test set statistics-----
R-squared of the model in test set is: 0.8149185869526183
Mean absolute error of the prediction is: 0.12770726087011366
Mean squared error of the prediction is: 0.03086152030253385
Root mean squared error of the prediction is: 0.17567447254092966
Mean absolute percentage error of the prediction is: 1.0685444897303116


### 5. Evaluate your best model on the test set.

In [37]:
# making predictions
y_preds = results.predict(X_test)


print(f"Mean absolute error of the prediction is: {mean_absolute_error(y_test, y_preds)}")
print(f"Mean squared error of the prediction is: {mse(y_test, y_preds)}")
print(f"Root mean squared error of the prediction is: {rmse(y_test, y_preds)}")
print(f"Mean absolute percentage error of the prediction is: {np.mean(np.abs((y_test - y_preds) / y_test)) * 100}")

Mean absolute error of the prediction is: 0.12570372872853813
Mean squared error of the prediction is: 0.029192121871305193
Root mean squared error of the prediction is: 0.17085702172080958
Mean absolute percentage error of the prediction is: 1.0503577667818464


From the new models constructed above, it can be seen that the OLS model performs better than the new models. The original OLS model has an R-Squared value of .832, which is higher than the R-Squared values found in all of the new models. Moreover, all of the error values in the OLS model are lower than the error values for the new models. Lower error values indicate better model performance, so on the basis of these metrics, the OLS model can be selected over the other models that were constructed.

### 6. So far, you have only used the features in the dataset. However, house prices can be affected by many factors like economic activity and the interest rates at the time they are sold. So, try to find some useful factors that are not included in the dataset. Integrate these factors into your model and assess the prediction performance of your model. Discuss the implications of adding these external variables into your model.

For the purposes of investigation, interest rates will be added to the initial dataset. Before the rates can be added, the dataset's timespan must be determined, because interest rates change multiple times a month.

In [39]:
housing_df.yrsold.max()

2010

In [43]:
housing_df.yrsold.min()

2006

In [51]:
housing_df[housing_df['yrsold']==2010].mosold.max()

7

In [54]:
housing_df[housing_df['yrsold']==2006].mosold.min()

1

The dataset has information from January of 2006 to July of 2010. The average monthly interest rate will be added to the dataset so that its effect on housing prices can be investigated. Adding this variable to our model will hopefully make it more accurate. If interest rate is highly correlated with another variable already in the model, the model may become less accurate. Additionally, since our data only covers the period from 2006 to 2010 - a time that saw great economic turmoil in the United States - we may not be seeing a completely accurate picture of either interest rates or housing prices.

In [63]:
housing_df['interest'] = 0

In [64]:
housing_df.loc[(housing_df['yrsold']==2006) & (housing_df['mosold']==1),'interest'] = 6.15
housing_df.loc[(housing_df['yrsold']==2006) & (housing_df['mosold']==2),'interest'] = 6.25
housing_df.loc[(housing_df['yrsold']==2006) & (housing_df['mosold']==3),'interest'] = 6.32
housing_df.loc[(housing_df['yrsold']==2006) & (housing_df['mosold']==4),'interest'] = 6.51
housing_df.loc[(housing_df['yrsold']==2006) & (housing_df['mosold']==5),'interest'] = 6.60
housing_df.loc[(housing_df['yrsold']==2006) & (housing_df['mosold']==6),'interest'] = 6.68
housing_df.loc[(housing_df['yrsold']==2006) & (housing_df['mosold']==7),'interest'] = 6.76
housing_df.loc[(housing_df['yrsold']==2006) & (housing_df['mosold']==8),'interest'] = 6.52
housing_df.loc[(housing_df['yrsold']==2006) & (housing_df['mosold']==9),'interest'] = 6.40
housing_df.loc[(housing_df['yrsold']==2006) & (housing_df['mosold']==10),'interest'] = 6.36
housing_df.loc[(housing_df['yrsold']==2006) & (housing_df['mosold']==11),'interest'] = 6.24
housing_df.loc[(housing_df['yrsold']==2006) & (housing_df['mosold']==12),'interest'] = 6.14

housing_df.loc[(housing_df['yrsold']==2007) & (housing_df['mosold']==1),'interest'] = 6.22
housing_df.loc[(housing_df['yrsold']==2007) & (housing_df['mosold']==2),'interest'] = 6.29
housing_df.loc[(housing_df['yrsold']==2007) & (housing_df['mosold']==3),'interest'] = 6.16
housing_df.loc[(housing_df['yrsold']==2007) & (housing_df['mosold']==4),'interest'] = 6.18
housing_df.loc[(housing_df['yrsold']==2007) & (housing_df['mosold']==5),'interest'] = 6.26
housing_df.loc[(housing_df['yrsold']==2007) & (housing_df['mosold']==6),'interest'] = 6.66
housing_df.loc[(housing_df['yrsold']==2007) & (housing_df['mosold']==7),'interest'] = 6.70
housing_df.loc[(housing_df['yrsold']==2007) & (housing_df['mosold']==8),'interest'] = 6.57
housing_df.loc[(housing_df['yrsold']==2007) & (housing_df['mosold']==9),'interest'] = 6.38
housing_df.loc[(housing_df['yrsold']==2007) & (housing_df['mosold']==10),'interest'] = 6.38
housing_df.loc[(housing_df['yrsold']==2007) & (housing_df['mosold']==11),'interest'] = 6.21
housing_df.loc[(housing_df['yrsold']==2007) & (housing_df['mosold']==12),'interest'] = 6.10

housing_df.loc[(housing_df['yrsold']==2008) & (housing_df['mosold']==1),'interest'] = 5.76
housing_df.loc[(housing_df['yrsold']==2008) & (housing_df['mosold']==2),'interest'] = 5.92
housing_df.loc[(housing_df['yrsold']==2008) & (housing_df['mosold']==3),'interest'] = 5.97
housing_df.loc[(housing_df['yrsold']==2008) & (housing_df['mosold']==4),'interest'] = 5.92
housing_df.loc[(housing_df['yrsold']==2008) & (housing_df['mosold']==5),'interest'] = 6.04
housing_df.loc[(housing_df['yrsold']==2008) & (housing_df['mosold']==6),'interest'] = 6.32
housing_df.loc[(housing_df['yrsold']==2008) & (housing_df['mosold']==7),'interest'] = 6.43
housing_df.loc[(housing_df['yrsold']==2008) & (housing_df['mosold']==8),'interest'] = 6.48
housing_df.loc[(housing_df['yrsold']==2008) & (housing_df['mosold']==9),'interest'] = 6.04
housing_df.loc[(housing_df['yrsold']==2008) & (housing_df['mosold']==10),'interest'] = 6.20
housing_df.loc[(housing_df['yrsold']==2008) & (housing_df['mosold']==11),'interest'] = 6.09
housing_df.loc[(housing_df['yrsold']==2008) & (housing_df['mosold']==12),'interest'] = 5.29

housing_df.loc[(housing_df['yrsold']==2009) & (housing_df['mosold']==1),'interest'] = 5.05
housing_df.loc[(housing_df['yrsold']==2009) & (housing_df['mosold']==2),'interest'] = 5.13
housing_df.loc[(housing_df['yrsold']==2009) & (housing_df['mosold']==3),'interest'] = 5.00
housing_df.loc[(housing_df['yrsold']==2009) & (housing_df['mosold']==4),'interest'] = 4.81
housing_df.loc[(housing_df['yrsold']==2009) & (housing_df['mosold']==5),'interest'] = 4.86
housing_df.loc[(housing_df['yrsold']==2009) & (housing_df['mosold']==6),'interest'] = 5.42
housing_df.loc[(housing_df['yrsold']==2009) & (housing_df['mosold']==7),'interest'] = 5.22
housing_df.loc[(housing_df['yrsold']==2009) & (housing_df['mosold']==8),'interest'] = 5.19
housing_df.loc[(housing_df['yrsold']==2009) & (housing_df['mosold']==9),'interest'] = 5.06
housing_df.loc[(housing_df['yrsold']==2009) & (housing_df['mosold']==10),'interest'] = 4.95
housing_df.loc[(housing_df['yrsold']==2009) & (housing_df['mosold']==11),'interest'] = 4.88
housing_df.loc[(housing_df['yrsold']==2009) & (housing_df['mosold']==12),'interest'] = 4.93

housing_df.loc[(housing_df['yrsold']==2010) & (housing_df['mosold']==1),'interest'] = 5.03
housing_df.loc[(housing_df['yrsold']==2010) & (housing_df['mosold']==2),'interest'] = 4.99
housing_df.loc[(housing_df['yrsold']==2010) & (housing_df['mosold']==3),'interest'] = 4.97
housing_df.loc[(housing_df['yrsold']==2010) & (housing_df['mosold']==4),'interest'] = 5.10
housing_df.loc[(housing_df['yrsold']==2010) & (housing_df['mosold']==5),'interest'] = 4.89
housing_df.loc[(housing_df['yrsold']==2010) & (housing_df['mosold']==6),'interest'] = 4.74
housing_df.loc[(housing_df['yrsold']==2010) & (housing_df['mosold']==7),'interest'] = 4.56


In [65]:
# Y is the target variable
# log transform the target variable to make it more normal
Y = np.log1p(housing_df['saleprice'])
# X is the feature set
X = housing_df[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalsf', 'int_over_sf', 'interest'] + dummy_column_names]

In [66]:
X = sm.add_constant(X)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)

results = sm.OLS(y_train, X_train).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.832
Model:                            OLS   Adj. R-squared:                  0.831
Method:                 Least Squares   F-statistic:                     477.8
Date:                Wed, 04 Dec 2019   Prob (F-statistic):               0.00
Time:                        07:51:20   Log-Likelihood:                 464.71
No. Observations:                1168   AIC:                            -903.4
Df Residuals:                    1155   BIC:                            -837.6
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           9.8684      0.109     90.266      0.000       9.654      10.083
overallqual     0.1890      0.009     20.076      0.000       0.171       0.207
grlivarea    9.536e-05   1.89e-05      5.051      0.000    5.83e-05       0.000
garagecars      0.0777      0.015      5.233      0.000       0.049       0.107
garagearea      0.0001   5.04e-05      2.139      0.033    8.94e-06       0.000
totalsf         0.0003   2.58e-05     11.112      0.000       0.000       0.000
int_over_sf -2.561e-05   3.02e-06     -8.485      0.000   -3.15e-05   -1.97e-05
interest        0.0083      0.007      1.187      0.235      -0.005       0.022
mszoning_FV     0.3922      0.065      6.072      0.000       0.265       0.519
mszoning_RH     0.2640      0.074      3.581      0.000       0.119       0.409
mszoning_RL     0.3880      0.060      6.483      0.000       0.271       0.505
mszoning_RM     0.2153      0.061      3.554      0.000       0.096       0.334
street_Pave    -0.0539      0.075     -0.722      0.471      -0.201       0.093
==============================================================================
Omnibus:                      350.606   Durbin-Watson:                   1.876
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2728.044
Skew:                          -1.165   Prob(JB):                         0.00
Kurtosis:                      10.116   Cond. No.                     5.42e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.42e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [67]:
# making predictions
y_preds = results.predict(X_test)


print(f"Mean absolute error of the prediction is: {mean_absolute_error(y_test, y_preds)}")
print(f"Mean squared error of the prediction is: {mse(y_test, y_preds)}")
print(f"Root mean squared error of the prediction is: {rmse(y_test, y_preds)}")
print(f"Mean absolute percentage error of the prediction is: {np.mean(np.abs((y_test - y_preds) / y_test)) * 100}")

Mean absolute error of the prediction is: 0.12533295688787052
Mean squared error of the prediction is: 0.028969043863888252
Root mean squared error of the prediction is: 0.17020294904580313
Mean absolute percentage error of the prediction is: 1.047293096836631


The addition of interest to the model has not improved the model by a large margin. The variable itself does not add a statistically significant amount of information to the model (p = .235), so it can be dropped without negatively affecting the model. The R-Squared in this model is the same as the R-Squared from the initial OLS model (.832), but the error metrics for the new model have gotten lower, indicating a slight increase in predictive performance. Altogether, it is clear that, unfortunately, the addition of interest rate data to the model does not significantly improve the model's power.